# Answer Key

Below are the answers to the exercises given in the companion notebooks but first let's setup the visualization options.


In [ ]:
%%graph_notebook_vis_options
{
  "groups": {    
    "person": {
      "color": "#9ac7bf"
    },
    "review": {
      "color": "#f8cecc"
    },
    "city": {
      "color": "#d5e8d4"
    },
    "state": {
      "color": "#dae8fc"
    },
    "review_rating": {
      "color": "#e1d5e7"
    },
    "restaurant": {
      "color": "#ffe6cc"
    },
    "cusine": {
      "color": "#fff2cc"
    }
  }
}

## 01-Basic-Read-Queries

### Exercise B-1 Find the first name of Dave's friends

In [ ]:
%%oc 

MATCH (:person {first_name: 'Dave'})-[:friends]->(f:person) 
RETURN f.first_name

### Exercise B-2 Find the first name of the friends of Dave's friends

In [ ]:
%%oc 

MATCH (:person {first_name:'Dave'})-[:friends]->()-[:friends]->(p:person) 
RETURN DISTINCT p.first_name

### Exercise B-3 Find out how the friends of Dave's friends are connected

In [ ]:
%%oc

MATCH p=(d:person {first_name:'Dave'})-[:friends]->()-[:friends]->(:person) 
RETURN p

### Exercise B-4  Which friends should we recommend for Dave?


In [ ]:
%%oc

MATCH p=(d:person {first_name:'Dave'})-[:friends]->()-[:friends]->(foff:person) 
WHERE NOT (foff)-[:friends]->(d)
RETURN DISTINCT foff.first_name

## 02-Variable-Length-Paths

### Exercise VLP-1 Find the first name of Dave's friends

In [ ]:
%%oc

MATCH (:person {first_name:'Dave'})-[:friends*2]->(p:person) 
RETURN DISTINCT p.first_name

### Exercise VLP-2 Find all `person` nodes connected to Dave

In [ ]:
%%oc

MATCH p=(:person {first_name:'Dave'})-[:friends*]->(:person) 
RETURN p

### Exercise VLP-3 Find if Dave and Denise are connected

In [ ]:
%%oc

MATCH p=(:person {first_name:'Dave'})-[:friends*1..]-(:person {first_name:'Denise'})
RETURN True 
LIMIT 1

### Exercise VLP-4 Find all the ways Dave and Denise are connected

In [ ]:
%%oc

MATCH p=(:person {first_name:'Dave'})-[:friends*1..]-(:person {first_name:'Denise'})
RETURN p 

## 03-Ordering-Functions-Grouping

### Exercise G-1 What are the 3 highest restaurants?

In [ ]:
%%oc
MATCH (r:restaurant)<-[:about]-(rev:review)
WITH r, avg(rev.rating) AS avg_rating
ORDER BY avg_rating DESC
LIMIT 3
MATCH (r)-[:serves]->(c:cuisine)
RETURN r.name, c.name, avg_rating
ORDER BY avg_rating Desc

### Exercise G-2 Find the top 3 highest rated restaurants in the city where Dave lives

In [ ]:
%%oc
MATCH (p:person {first_name: 'Dave'})-[:lives]->(:city)<-[:within]-(r:restaurant)<-[:about]-(v:review)
WITH r, avg(v.rating) AS rating_average, p
RETURN r.name AS name,
r.address AS address, rating_average
ORDER BY rating_average DESC 
LIMIT 3

### Exercise G-3 What Mexican or Chinese restaurant near Dave that is the highest rated?

In [ ]:
%%oc
MATCH (p:person {first_name: 'Dave'})-[:lives]->(:city)<-[:within]-(r:restaurant)-[:serves]->(c:cuisine) 
WHERE c.name IN ['Mexican', 'Chinese'] 
WITH r
MATCH (r)<-[:about]-(v:review) 
WITH r, avg(v.rating) AS rating_average 
RETURN r.name AS name, 
    r.address AS address, rating_average
ORDER BY rating_average DESC 
LIMIT 1

### Exercise G-4 What are the top 3 restaurants, recommended by his friends, where Dave lives? (Personalized Recommendation)

In [ ]:
%%oc
MATCH (p:person {first_name: 'Dave'})-[:lives]->(c:city)
MATCH (p)-[:friends]-()-[:wrote]-(v)-[:about]-
(r:restaurant)-[:within]-(c) 
RETURN  r.name AS name, r.address AS address, avg(v.rating) as rating_average
ORDER BY rating_average DESC
LIMIT 3

## 04-Creating-Updating-Delete-Queries

###  Exercise M-1 Create a new person `Leonhard Euler`  and connect them to `Dave`?

In [ ]:
%%oc
CREATE (le:person {first_name: 'Leonhard', last_name: 'Euler'})
WITH le
MATCH (dave:person {first_name: 'Dave'})
CREATE (le)-[:friends]->(dave)
RETURN le

### Exercise M-2 Upsert the a list of followers and add an edge to `Dave`?

In [ ]:
%%oc
WITH [{first_name: 'Taylor', last_name: 'Hall'},{first_name: 'Kelvin', last_name: 'Fernsby'},{first_name: 'Ian', last_name: 'Rochester'}] as followers

UNWIND followers as f
MERGE (d:person {first_name: 'Dave'})
MERGE (n:person {first_name: f.first_name, last_name: f.last_name})-[r:follower]->(d)
ON CREATE
    SET r.creation='Created'
ON MATCH
    SET r.creation='Updated'
RETURN r

### Exercise M-3 Delete all `followers` edges and remove any connected nodes with no other edges?

In [ ]:
%%oc
MATCH (a)-[r:followers]-()
DELETE r
WITH collect(DISTINCT a) as a, count(r) as edge_cnt
UNWIND a as n
MATCH (n)
WHERE size((n)--()) = 0
DELETE n
WITH count(n) as node_cnt, edge_cnt
RETURN node_cnt, edge_cnt